# Prompt Engineering 101: Learn by Building

This notebook demonstrates key prompt engineering concepts using Google's Gemini API.

In [ ]:
# Install required packages
%pip install google-genai python-dotenv

In [2]:
from google import genai
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize Gemini client
client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

print("✅ Gemini API configured successfully!")

✅ Gemini API configured successfully!


In [3]:
from google.genai import types
def generate_response(prompt, system_prompt=None, model_id="gemini-2.5-flash"):
    """
    Generate response using Google Gemini API with optional system instruction
    
    Args:
        prompt: User message or main prompt
        system_instruction: Optional system instruction to set AI behavior
        model_id: Gemini model to use
    """
    # Create config with system instruction if provided
    config = None
    if system_prompt:
        config = types.GenerateContentConfig(
            system_instruction=system_prompt
        )
    
    response = client.models.generate_content(
        model=model_id,
        contents=prompt,
        config=config
    )
    return response.text

print("✅ Helper function with system instruction support defined!")

✅ Helper function with system instruction support defined!


## 1. System and User Prompts

System prompts set the behavior and context for the AI, while user prompts contain the actual requests.

In [4]:
# Example: Customer Service Bot with System Instruction Parameter

# Define system instruction
system_prompt = """
You are a helpful customer service representative for TechStore, an electronics retailer.
Always be polite, professional, and solution-oriented.
Keep responses concise but helpful.
"""

# Test with user message using system instruction parameter
user_message = "I bought a laptop last week and it won't turn on. What should I do?"

In [5]:
print("\n🔄 No system prompt:")
basic_response = generate_response(user_message)
print(basic_response)


🔄 No system prompt:
That's incredibly frustrating, especially with a brand new laptop! Don't panic, there are several things you can try. Since you just bought it, you're definitely covered by warranty, but let's go through some common troubleshooting steps first.

**Before you do anything else, make sure you keep all packaging, receipts, and documentation. You'll need these for a return or warranty claim if it comes to that.**

Here's what you should do, in order from simplest to more involved:

---

### Basic Checks & Troubleshooting

1.  **Check the Power Source (Most Common Issue!):**
    *   **Is the charger plugged in correctly?** Make sure both ends of the charger cable are firmly connected: one into the laptop's charging port and the other into a wall outlet.
    *   **Is the wall outlet working?** Plug another device (like a phone charger or a lamp) into the same outlet to confirm it has power. If not, try a different outlet.
    *   **Check the charger's light:** Many laptop

In [6]:
print("\n✨ With system prompt:")
response = generate_response(user_message, system_prompt)
print(response)


✨ With system prompt:
I'm sorry to hear your new laptop isn't turning on! That's definitely frustrating.

First, please ensure it's properly plugged into a working power outlet and the power adapter is securely connected to the laptop.

If it still doesn't power on, we can certainly help. Since you just purchased it last week, it's likely covered under our return or warranty policy. Please bring your laptop and proof of purchase to any TechStore location, or contact our customer support team directly, and we'll be happy to assist you with a solution.


💡 The first response (without system prompt): very long, costs more, and takes more time (15 seconds). It is also less helpful as it's less concise.

💡 The second response (with system prompt): concise and helpful, took only 2 seconds. It is also cheaper.

## 2. Chat History Management

Maintaining conversation context by keeping track of the chat history.

First, let's see what happens when we don't keep track of the chat history

In [7]:
print("💬 Conversation Example:")
print("\n--- Message 1 ---")
response1 = generate_response("My computer is running slowly", system_prompt)
print(f"User: My computer is running slowly")
print(f"Bot: {response1}")

print("\n--- Message 2 ---")
response2 = generate_response("It's a Windows laptop, about 2 years old", system_prompt)
print(f"User: It's a Windows laptop, about 2 years old")
print(f"Bot: {response2}")

💬 Conversation Example:

--- Message 1 ---
User: My computer is running slowly
Bot: I understand that can be frustrating. Here are a few common steps that often help:

1.  **Restart your computer:** This can clear temporary issues.
2.  **Check for updates:** Ensure your operating system and drivers are current.
3.  **Close unnecessary programs:** Too many open applications can slow things down.
4.  **Check disk space:** A nearly full hard drive can impact performance.

If these steps don't help, we can explore more detailed troubleshooting or discuss potential service options.

--- Message 2 ---
User: It's a Windows laptop, about 2 years old
Bot: Thanks for providing that detail about your Windows laptop.

To help me assist you further, could you please tell me what issue you're experiencing or what you'd like help with today?


---
💡 As you can see, when answering the second query, model didn't have any previous context and couldn't provide a coherent response.

---
Now let's try using chat history, so the model will "remember" previous interactions and will provide more coherent responses.

_Important Note_: Gemini has special [chat interface](https://ai.google.dev/gemini-api/docs/text-generation#multi-turn-conversations) in their API that implements this logic. For transparency we will implement it from scratch using python classes and store chat history as a list.

In [8]:
class ChatBot:
    def __init__(self, system_prompt):
        self.system_prompt = system_prompt
        self.chat_history = []
    
    def send_message(self, user_message):
        # Add user message to history
        self.chat_history.append(f"User: {user_message}")
        
        # Build full conversation context
        conversation =  "\n".join(self.chat_history) + "\nAssistant:"
        
        # Get response
        assistant_response = generate_response(conversation, self.system_prompt)
        
        # Add assistant response to history
        self.chat_history.append(f"Assistant: {assistant_response}")
        
        return assistant_response

In [9]:
# Create a chatbot instance with the system prompt
bot = ChatBot(system_prompt)

# Simulate a conversation
print("💬 Conversation Example:")
print("\n--- Message 1 ---")
response1 = bot.send_message("My computer is running slowly")
print(f"User: My computer is running slowly")
print(f"Bot: {response1}")

print("\n--- Message 2 ---")
response2 = bot.send_message("It's a Windows laptop, about 2 years old")
print(f"User: It's a Windows laptop, about 2 years old")
print(f"Bot: {response2}")

💬 Conversation Example:

--- Message 1 ---
User: My computer is running slowly
Bot: I understand how frustrating a slow computer can be!

Often, a simple restart can resolve this. If that doesn't help, you might try:
1.  **Closing unused programs and browser tabs.**
2.  **Checking for system updates.**
3.  **Running a quick virus scan.**

Please let me know if you've tried these or if the issue persists, and I'll be happy to provide further assistance.

--- Message 2 ---
User: It's a Windows laptop, about 2 years old
Bot: Thank you for that additional information. A 2-year-old Windows laptop might benefit from a few specific optimizations:

1.  **Run Disk Cleanup:** Search for "Disk Cleanup" in Windows and run it to remove temporary files and other unnecessary data.
2.  **Check Startup Programs:** Open Task Manager (Ctrl+Shift+Esc), go to the "Startup" tab, and disable any non-essential programs from launching with Windows.
3.  **Update Drivers:** Ensure your graphics and other key dri

---
💡 _Note:_ As you can see, response to the second message is now coherent because we've provided relevant context to the model.

## 3. Task Decomposition (Chaining)

Breaking complex tasks into smaller, manageable steps for better reliability.

In [10]:
# Example: Processing a refund request through multiple steps

def step1_identify_items(customer_request):
    """Step 1: Identify which items need to be refunded"""
    prompt = f"""
    Extract the items that the customer wants to refund from this request.
    Return only the item names, one per line.
    
    Customer request: {customer_request}
    
    Items to refund:
    """
    
    return generate_response(prompt).strip()

def step2_check_eligibility(items, purchase_date):
    """Step 2: Check refund eligibility"""
    prompt = f"""
    Check if these items are eligible for refund based on our 30-day return policy.
    Purchase date: {purchase_date}
    Items: {items}
    
    For each item, respond with: [Item name] - ELIGIBLE or NOT ELIGIBLE (reason)
    """
    
    return generate_response(prompt).strip()

def step3_generate_instructions(eligibility_results):
    """Step 3: Provide refund instructions"""
    prompt = f"""
    Based on this eligibility check, provide clear next steps for the customer.
    Be helpful and professional.
    
    Eligibility results: {eligibility_results}
    
    Next steps:
    """
    
    return generate_response(prompt).strip()

In [11]:
# Example usage
customer_request = "I want to return my wireless headphones and phone case I bought 2 weeks ago. The headphones don't fit well."
purchase_date = "2024-12-15"

print("🔄 Task Decomposition Example:")
print(f"Customer Request: {customer_request}")
print("\n--- Step 1: Identify Items ---")
items = step1_identify_items(customer_request)
print(items)

print("\n--- Step 2: Check Eligibility ---")
eligibility = step2_check_eligibility(items, purchase_date)
print(eligibility)

print("\n--- Step 3: Generate Instructions ---")
instructions = step3_generate_instructions(eligibility)
print(instructions)

🔄 Task Decomposition Example:
Customer Request: I want to return my wireless headphones and phone case I bought 2 weeks ago. The headphones don't fit well.

--- Step 1: Identify Items ---
wireless headphones
phone case

--- Step 2: Check Eligibility ---
To determine eligibility, we need to know the *current date* you are checking this.

The 30-day return window for items purchased on 2024-12-15 ends on **2025-01-14**.

Assuming the items are otherwise in returnable condition (original packaging, no damage, etc.), here's the eligibility based on when the return request is made:

*   **Wireless headphones** - ELIGIBLE (if the return request is made on or before 2025-01-14) or NOT ELIGIBLE (if the return request is made after 2025-01-14, as the 30-day window has closed)
*   **Phone case** - ELIGIBLE (if the return request is made on or before 2025-01-14) or NOT ELIGIBLE (if the return request is made after 2025-01-14, as the 30-day window has closed)

--- Step 3: Generate Instructions ---

---
💡 Breaking down complex tasks, makes model less prone to errors and makes it easier to debug such workflows.
**Divide and conquer.**

## 4. Few-Shot Prompting (In-Context Learning)

Providing examples to guide the model's behavior and output format.

In [12]:
# Example: Sentiment analysis with few-shot prompting

def analyze_sentiment_few_shot(customer_message):
    prompt = f"""
    Analyze the sentiment of customer messages and categorize them as POSITIVE, NEGATIVE, or NEUTRAL.
    Also provide a brief reason.
    
    Examples:
    
    Message: "I love this product! It works perfectly and arrived quickly."
    Sentiment: POSITIVE
    Reason: Customer expresses satisfaction with product quality and delivery
    
    Message: "The item is okay, nothing special but does what it's supposed to do."
    Sentiment: NEUTRAL
    Reason: Customer shows neither strong satisfaction nor dissatisfaction
    
    Message: "Terrible quality! Broke after one day and customer service was rude."
    Sentiment: NEGATIVE
    Reason: Customer reports product failure and poor service experience
    
    Now analyze this message:
    Message: "{customer_message}"
    Sentiment:
    Reason:
    """
    
    return generate_response(prompt).strip()

In [13]:
# Test with different messages
test_messages = [
    "The delivery was delayed but the product quality is amazing!",
    "I'm not sure if this is worth the price, it's just average",
    "Worst purchase ever! Complete waste of money!"
]

print("🎯 Few-Shot Prompting Example:")
for i, message in enumerate(test_messages, 1):
    print(f"\n--- Test {i} ---")
    print(f"Message: {message}")
    result = analyze_sentiment_few_shot(message)
    print(result)

🎯 Few-Shot Prompting Example:

--- Test 1 ---
Message: The delivery was delayed but the product quality is amazing!
Sentiment: POSITIVE
Reason: Customer expresses strong satisfaction with the product quality, which outweighs the negative experience of delayed delivery.

--- Test 2 ---
Message: I'm not sure if this is worth the price, it's just average
Sentiment: NEUTRAL
Reason: Customer expresses uncertainty about value and describes the item as ordinary or satisfactory, showing no strong positive or negative sentiment.

--- Test 3 ---
Message: Worst purchase ever! Complete waste of money!
Sentiment: NEGATIVE
Reason: Customer uses strong negative language ("worst purchase," "complete waste of money") to express extreme dissatisfaction and regret.


---
💡 With examples of input/output pairs we can better steer the model's behaviour and output format.

## 5. Chain-of-Thought Prompting

Encouraging the model to show its reasoning process step by step.

In [14]:
# Example: Troubleshooting with chain-of-thought reasoning

def troubleshoot_with_reasoning(problem_description):
    prompt = f"""
    Help troubleshoot this technical problem. Think through it step by step and show your reasoning.
    
    Example of good reasoning:
    Problem: "My phone won't charge when I plug it in"
    
    Let me think through this step by step:
    1. First, I need to identify possible causes: faulty cable, damaged port, dead battery, or software issue
    2. The most common cause is usually the charging cable or adapter
    3. Next most likely is debris in the charging port
    4. Less likely but possible: battery completely dead or software glitch
    
    Based on this reasoning, here's what to try:
    - Try a different charging cable and adapter first
    - Check the charging port for lint or debris
    - Try a different power outlet
    - If none work, try a force restart
    
    Now solve this problem using the same step-by-step approach:
    Problem: "{problem_description}"
    
    Let me think through this step by step:
    """
    
    return generate_response(prompt).strip()

In [15]:

# Test with a technical problem
problem = "My laptop keeps freezing when I try to open multiple applications"

print("🧠 Chain-of-Thought Example:")
print(f"Problem: {problem}")
print("\nAI Reasoning:")
solution = troubleshoot_with_reasoning(problem)
print(solution)

🧠 Chain-of-Thought Example:
Problem: My laptop keeps freezing when I try to open multiple applications

AI Reasoning:
Let me think through this step by step:

1.  **First, I need to identify possible causes:** When a laptop freezes opening multiple applications, it strongly suggests a resource limitation or software conflict.
    *   **Resource Exhaustion:** The most common culprit. This means the laptop doesn't have enough RAM (memory) or CPU (processor) power to handle all the applications simultaneously.
    *   **Overheating:** When the CPU or GPU gets too hot, they automatically slow down (throttle) or even shut down to prevent damage, which can manifest as freezing.
    *   **Outdated/Corrupt Drivers:** Graphics drivers, chipset drivers, or other system drivers can cause instability when under load.
    *   **Too Many Background Processes:** Even if not explicitly opened, many programs run in the background, consuming resources.
    *   **Malware/Viruses:** Malicious software can

---
💡  Asking the model to reason through a task step by step significantly improves its performance, especially on reasoning-heavy tasks.

## Key Takeaways

1. **System Prompts**: Set clear behavior and context upfront
2. **Chat History**: Maintain conversation context for coherent interactions
3. **Task Decomposition**: Break complex tasks into manageable steps
4. **Few-Shot Prompting**: Use examples to guide model behavior
5. **Chain-of-Thought**: Encourage step-by-step reasoning for better results

## Next Steps

- Experiment with different system prompts
- Try combining techniques for your specific use case
- Test with real user scenarios
- Monitor and iterate based on performance

Remember: **Learn by building!** 🚀